# Groupby apply on large (relational) data set

In [66]:
import pandas as pd
import numpy as np

from datetime import datetime

from scipy import signal

pd.set_option('display.max_rows',500)

In [49]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv', sep=';' ,parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date', ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()


,date,state,country,confirmed
0,2020-01-22,NaN,Afghanistan,0.0
1,2020-01-22,Zhejiang,Cuba,0.0
2,2020-01-22,Zhejiang,Croatia,0.0
3,2020-01-22,Zhejiang,Cote d'Ivoire,0.0
4,2020-01-22,Zhejiang,Costa Rica,0.0


In [50]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')| (pd_JH_data['country']=='Germany'))& (pd_JH_data['date']>'2020-07-20')]

In [51]:
test_data

,date,state,country,confirmed
81846,2020-07-21,Zhejiang,Germany,203717.0
81965,2020-07-21,Zhejiang,US,3899211.0
82055,2020-07-21,NaN,Germany,203717.0
82175,2020-07-21,NaN,US,3899211.0
82299,2020-07-22,Zhejiang,Germany,204276.0
82418,2020-07-22,Zhejiang,US,3970121.0
82507,2020-07-22,NaN,Germany,204276.0
82627,2020-07-22,NaN,US,3970121.0
82749,2020-07-23,Zhejiang,Germany,204881.0
82871,2020-07-23,Zhejiang,US,4038816.0


In [52]:
test_data.groupby(['country']).agg(np.max)

,date,confirmed
country,,
Germany,2020-09-06,251728.0
US,2020-09-06,6276365.0


In [53]:
# %load ../src/features/build_features.py
import numpy as np
from sklearn import linear_model


reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate
        Parameters:
        ----------
        in_array : pandas.series
        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

    

the test slope is: [2.]


In [54]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state    country                      
Zhejiang Germany 2020-09-06   251728.0
         US      2020-09-06  6276365.0

In [55]:
def rolling_reg(df_input,col='confirmed'):
    
    days_back=3
    result=df_input[col].rolling(
                       window=days_back,
                       min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [56]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg, 'confirmed')

state     country        
Zhejiang  Germany  81846            NaN
                   82299            NaN
                   82749     351.016037
                   83201     304.271220
                   83654     294.336435
                   84105     394.998723
                   84557     495.649081
                   85010     398.388462
                   85463     289.802417
                   85913     228.223195
                   86365     226.112610
                   86818     286.140136
                   87270     513.701989
                   87721     382.360458
                   88175     263.747512
                   88625     212.804529
                   89079     193.571536
                   89529     206.544407
                   89981     231.809013
                   90435     397.061661
                   90888     271.110696
                   91338     194.001184
                   91790     186.844463
                   92244     161.177186
              

In [57]:
pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg, 'confirmed')

state     country         
Alberta   Canada    351              NaN
                    801              NaN
                    1252             NaN
                    1704             NaN
                    2157             NaN
                                 ...    
Zhejiang  Zimbabwe  101390     93.115839
                    101811    111.344538
                    102294     67.514238
                    102684     85.333333
                    103507           inf
Name: confirmed, Length: 60914, dtype: float64

In [58]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg, 'confirmed').reset_index()

In [60]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR', 'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,351,NaN
1,Alberta,Canada,801,NaN
2,Alberta,Canada,1252,NaN
3,Alberta,Canada,1704,NaN
4,Alberta,Canada,2157,NaN


In [62]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,level_0,index,date,state,country,confirmed
0,0,0,2020-01-22,NaN,Afghanistan,0.0
1,1,1,2020-01-22,Zhejiang,Cuba,0.0
2,2,2,2020-01-22,Zhejiang,Croatia,0.0
3,3,3,2020-01-22,Zhejiang,Cote d'Ivoire,0.0
4,4,4,2020-01-22,Zhejiang,Costa Rica,0.0


In [64]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')

In [65]:
pd_result_larg.head()

,level_0,index,date,state,country,confirmed,confirmed_DR
0,0,0,2020-01-22,NaN,Afghanistan,0.0,NaN
1,1,1,2020-01-22,Zhejiang,Cuba,0.0,NaN
2,2,2,2020-01-22,Zhejiang,Croatia,0.0,NaN
3,3,3,2020-01-22,Zhejiang,Cote d'Ivoire,0.0,NaN
4,4,4,2020-01-22,Zhejiang,Costa Rica,0.0,NaN


# Filtering the Data with Groupby apply !

In [67]:
def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result

In [68]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [69]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,level_0,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,0,2020-01-22,NaN,Afghanistan,0.0,NaN,NaN
1,1,1,2020-01-22,Zhejiang,Cuba,0.0,NaN,0.0
2,2,2,2020-01-22,Zhejiang,Croatia,0.0,NaN,0.0
3,3,3,2020-01-22,Zhejiang,Cote d'Ivoire,0.0,NaN,0.0
4,4,4,2020-01-22,Zhejiang,Costa Rica,0.0,NaN,0.0


# Filtered Doubling Rate


In [70]:
pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
60909,Zhejiang,Zimbabwe,101390,94.218077
60910,Zhejiang,Zimbabwe,101811,86.746841
60911,Zhejiang,Zimbabwe,102294,108.505933
60912,Zhejiang,Zimbabwe,102684,121.571129
60913,Zhejiang,Zimbabwe,103507,122.461400


In [71]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,level_0,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
103503,103503,103503,2020-09-06,NaN,San Marino,716.0,NaN,NaN,NaN
103504,103504,103504,2020-09-06,NaN,Saint Vincent and the Grenadines,61.0,NaN,NaN,NaN
103505,103505,103505,2020-09-06,NaN,Saint Lucia,26.0,NaN,NaN,NaN
103506,103506,103506,2020-09-06,NaN,Timor-Leste,27.0,NaN,NaN,NaN
103507,103507,103507,2020-09-06,Zhejiang,Zimbabwe,6837.0,inf,6876.8,122.4614


In [72]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)

In [73]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,level_0,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
102396,102396,102396,2020-09-04,NaN,Germany,250283.0,NaN,NaN,NaN
102639,102639,102639,2020-09-05,Zhejiang,Germany,251058.0,225.482717,250949.2,224.297394
102848,102848,102848,2020-09-05,NaN,Germany,251058.0,NaN,NaN,NaN
103090,103090,103090,2020-09-06,Zhejiang,Germany,251728.0,347.436678,252034.4,231.246959
103298,103298,103298,2020-09-06,NaN,Germany,251728.0,NaN,NaN,NaN


In [74]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)